In [2]:
import os
import warnings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import mysql.connector as sql
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

dotenv_path = "../.env"
load_dotenv(dotenv_path)

True

In [3]:
# make a connection to the database
dwh = sql.connect(
    host="127.0.0.1",   # ip-adres
    port=3306,   # poortnummer
    user=os.getenv("USER"),   # username
    password=os.getenv("PASSWORD"),   # wachtwoord
    database="AirFaresDWH"   # naam van de datawarehouse in mysql workbench
    # database="groep8dep"   # naam van de OLTP database in mysql workbench
)

# check if the connection succeeded
print(dwh)

In [11]:
# create query
query = """
select flight_key, depatureAirportKey, da.airport_name as departureAirport_name, arrivalAirportKey, aa.airport_name as arrivalAirport_name, departureDateKey, dd.date as departureDate, arrivalDateKey, availableSeats, price from factflightfare ff
join dimdate dd on dd.date_key = ff.departureDateKey
join dimdate de on de.date_key = ff.scrapeDateKey
join dimairport da on da.airport_key = ff.depatureAirportKey
join dimairport aa on aa.airport_key = ff.arrivalAirportKey;
"""

# create the dataframe while reading query
df = pd.read_sql(query, dwh)

# show all columns
pd.options.display.max_columns = None

# show the df
df

,flight_key,depatureAirportKey,departureAirport_name,arrivalAirportKey,arrivalAirport_name,departureDateKey,departureDate,arrivalDateKey,availableSeats,price
0,1153,7,Aereport De Chaleroi Bruxelles Sud,9,Heraklion Airport,20230523,2023-05-23,20230523,2,87.79
1,982,7,Aereport De Chaleroi Bruxelles Sud,8,Gago Coutinho Airport,20230623,2023-06-23,20230623,1,77.74
2,726,7,Aereport De Chaleroi Bruxelles Sud,4,Salento Airport,20230627,2023-06-27,20230627,-1,37.77
3,510,7,Aereport De Chaleroi Bruxelles Sud,2,Alicante Airport,20230702,2023-07-02,20230702,-1,134.12
4,1590,7,Aereport De Chaleroi Bruxelles Sud,14,Palma de Mallorca Airport,20230717,2023-07-17,20230717,-1,181.49
...,...,...,...,...,...,...,...,...,...,...
114260,2879,3,Antwerpen International Airport,2,Alicante Airport,20230711,2023-07-11,20230711,-1,219.99
114261,4015,5,Brussels Airline,12,Aeroporto di Napoli,20230922,2023-09-22,20230922,-1,149.99
114262,2933,3,Antwerpen International Airport,2,Alicante Airport,20230924,2023-09-24,20230924,-1,129.99
114263,4619,5,Brussels Airline,17,Tenerife South Airport,20231004,2023-10-04,20231004,-1,204.99


## Vraag 12: Hoeveel dagen voor vertrek is een vlucht volgeboekt? Zijn er bestemmingen waarbij de vluchten bijna volgeboekt zijn?

In [1]:
# 1ste vraag
# datum van vandaag of scrape date (VRAGEN)
# vertrek datum
# verschil tussen de 2 als er geen zitplaatsen vrij zijn

def calc_date_diff(row):
    if row['availableSeats'] == 0:
        today = dt.date.today()
        return (row['departureDate'] - today).days # row['scrapeDate']
    else:
        return "seats available"

df['countOfDaysBeforeMaxCapacity'] = df.apply(calc_date_diff, axis=1)
df


NameError: name 'df' is not defined

In [ ]:
# 2de vraag
# Zijn er bestemmingen waarbij de vluchten bijna volgeboekt zijn?
# powerbi (vraag12p2)
